# <center> Airplane Passenger Load Prediction <center>
### <center> Tommy TRAN, Thomas de MAREUIL - RAMP Submission: _2Tomsbetterthan1_ <center>


## 1. Context and objective

This report is our final project for the class _MAP536 - Python for Data Science_.

Our objective was to **forecast airplane passengers load** in the United States. We were initially provided with:
* training data : information about US domestic flights between 2011 and 2013
* external data : meteorological information about US airports

Based on these first pieces of information, we identified several steps to work on and improve our predictions:

1. **Feature engineering**:  
    $\to$ look for additional external data  
    $\to$ clean it and merge it with preexisting data  
    $\to$ adjust data encoding  
    $\to$ check features relevance  
<br>
2. **Model selection and tuning:**  
    $\to$ try several models and select the best-performing    
    $\to$ try model averaging and stacking  
    $\to$ tune model hyperparameters with gridsearch  
<br>

In this report, we will go through these steps and present you the ideas we got, the techniques we tried, the models we explored and the results we obtained.

## 2. Data Exploration and Feature Engineering

### A. Structure of the initial datasets

The initial **training dataset** contained information about 8902 US domestic flights: date (from 09/01/2011 to 03/05/2013), departure and arrival airports (20 different airports for both), how early tickets were booked (in weeks) and the number of passengers (our predicted variable).  

$\rightarrow$  This dataset contained no missing values and low skewness. Based on correlation plots we observed that `std_wtd` (standard deviation of weeks to departure) is strongly correlated to `WeeksToDeparture` and little correlated to `passengers`, therefore we will not to use it in the regression.

The initial **external dataset** contained information about meteorological features for the 20 airports, during the corresponding time period. Each line represents the property of an airport at a given date.  

$\rightarrow$ Based on correlation, we kept only the Mean values for each feature that also had Min and Max values (i.e. `Temperature`, `Humidity`, `Wind`, etc.).

### B. Adding more external data

We tried to identify the determinants of airplane passenger load, and after online research we came out with:

_General / time-related data:_
* **Holidays**: we added a binary indicator to identify holidays (ex: Matin Luther King Day, July 4th, Thanksgiving, etc.), with more plane traffic.  
* **Plane load factor**: percentage of empty seats in US domestic flights, obtained from the US Bureau of Transportation Services (BTS) database. We only found the average value per month, over the whole US market, which isn't very precise but gives and idea of plane load tendencies over the observed time period.  
* **Jet Fuel Prices**: found in the IATA (International Air Transport Association) database, jet fuel prices tend to impact the price of tickets and therefore passenger load.

_Data per airport:_
* **Airports ranking** and **frequentation** in terms of commercial passengers per year (IATA).  
* **City population and density** (IATA).  
* **Latitude and longitude (distances)**: the IATA database gave us latitude and longitude coordinates of the airports. We then explored the pyproj and geopy packages to compute distances, but we finally chose to use direct computation with the Haversine distance formula, which we embedded in the feature extractor.

_Data per flight route:_
* **Air time** (BTS): time spent in flight to link 2 airports.  
* **Average ticket prices** (US BTS): average ticket prices between 2 given US airports, which has a direct impact on passenger load. We did not find day by day values, but we obtained an averafe price per quarter over the observed period, for each of the relevant flight routes.  
* **Average daily number of passengers** for a given flight route (BTS): similarly, we found a quarter-by-quarter value of the daily number of passengers on the relevant flight routes, during the observed period.

### C. Data pre-processing: cleaning, merging, encoding

__*Producing a clean external dataset*__

A tricky part was first to clean the data we found online, and then to merge it into a single `external_data` file, including new and pre-existing external data (as we needed to have a single file to upload on RAMP).  

To clean it and format it, we used mainly `pandas`, `numpy` and `datetime`. We had, for example, to drop and select values, to perform computations on columns, to match city names with airport codes, to change data types, to manipulate dates and times, to reindex rows, etc. As we found extensive data online, we were not confronted with the problem of missing values.  

In the end we produced 4 external datasets to be merged together into a single file: the original weather data (sorted out by airport and date), our new general data (holidays, fuel prices...), our new data per airport (airport ranking, latitude, longitude, city population...) and our new data per flight route (flight time, average daily number of passengers...). 

Weather data, airport data and general data were easy to merge together on the `airport` and `date` columns (which they had in common), using the `pd.merge` function. Merging was more complicated for the fligh route data, as information _didn't correspond to a single airport_. As we could not keep 2 separated files, we just **concatenated** our data per flight route next to the rest of the data (using the `pd.concat`function), to produce a final `external_data` file.

##### __*Merging with the training dataset*__

We built the feature extractor in order to select features from our final `external_data` and merge them with the training data.

The training data is sorted out by flight route, so part of our external data (the flight route data) was easy to merge. We just had to add a flight route column in the training data, select the external data related to flight routes, and perform the merge on the new route column. We also split the dates in the training dataset using `datetime` in order to have months/quarters appear and to merge on the correct dates.  

For the other features, related to single airports, we had to merge them _twice_ with training data, based on the 2 airports in the itinerary (once on the Departure airport, once on the Arrival airport). Our feature extractor adds `_dep` and `_arr` suffixes in the column names in order to distinguish information related to departure and arrival airports. Once the merger done, we used the Haversine formula to compute distances between latitude and longitude coordinates of departure and arrival airports.

__*Encoding categorical features*__

Before feeding the model, we still had to encode the categorical features, i.e. dates (split into year, month, week, day, weekday) and airports (Dep and Arr). We tried several encoding methods, especially **target encoding** and **one-hot encoding**. Target encoding did not yield efficient results, therefore we chose to stick to one-hot encoding.

__*Drop irrelevant features*__



We made several _back-and-forths_ with the model to test differents sets of features and assess them with **feature importance analysis**. We then included a line in the feature extractor to drop the least significant features, and produce our final `X_encoded` file, to be fed to the model:

In [10]:
print(X_encoded.shape)
X_encoded.head()

(8902, 119)


,WeeksToDeparture,Mean TemperatureC,Rank_2018_Dep,population_Dep,density_Dep,Fuel_price_Dep,Holiday_Dep,LOAD_FACTOR_Dep,2018_freq_Dep,Rank_2018_Arr,population_Arr,density_Arr,2018_freq_Arr,Distance,Weekend,n_days,daily_passengers,average_fare,AIR_TIME_MEAN,d_ATL,d_BOS,d_CLT,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_ORD,d_PHL,d_PHX,d_SFO,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHX,a_SFO,y_2011,y_2013,m_1,m_3,m_4,m_5,m_6,m_8,m_9,m_10,m_11,m_12,d_1,d_2,d_3,d_4,d_5,d_7,d_8,d_9,d_10,d_12,d_18,d_23,d_25,d_27,d_28,d_29,d_30,d_31,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_14,w_20,w_21,w_22,w_26,w_27,w_35,w_36,w_37,w_38,w_41,w_44,w_46,w_47,w_49,w_50,w_51,w_52,q_1,q_2,q_3
0,12.875000,29,3,8675982.0,4612.0,2.719,0,86.62,39874879,4,5733259.0,1524.0,32800721,1285.250403,0.0,15510,3220.0,228.67,7560.62,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,14.285714,25,9,2073045.0,1747.0,3.247,0,80.64,23655285,5,2787266.0,1774.0,31363573,990.537032,0.0,15593,2286.0,123.86,4407.42,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,10.863636,19,5,2787266.0,1774.0,3.158,0,84.23,31363573,2,12815475.0,3295.0,42626783,1354.641049,0.0,15618,4379.0,172.05,5667.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,11.480000,19,1,5228750.0,1384.0,2.881,0,83.63,51866464,3,8675982.0,4612.0,39874879,942.326564,1.0,15256,2746.0,193.12,3693.83,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,11.450000,12,5,2787266.0,1774.0,3.237,0,78.90,31363573,7,3603761.0,7286.0,27794154,1536.901253,0.0,15391,2681.0,182.97,5300.04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## 3. Prediction

### A. Model Selection

We tried and evaluated several regression models locally, in order to obtain the lowest error rate on our testing set. Here are the results of our different tries, ranked from lowest to highest test rmse:

* **Exterme Gradient Boosting (`XGBoost`)**: the best-performing model. This is **the model that we finally selected**.  

```
----------------------------
train rmse = 0.137 ± 0.0025
test rmse = 0.267 ± 0.0072
```


* **Gradient Boosting**: 2nd best performer, just a litle behind `XGBoost`.
* **Random Forest**: the initial model in the starting kit, outperformed by gradient-based models.
* **Lasso Regression**: this method and the subsequent ones were significantly less performant.
* **Elastic Net Regression**
* **AdaBoost**
* **Linear Regression**

**_Model averaging / stacking_**  

We tried **averaging** the 3 best-performing models (XGBoost, GradientBoosting and RandomForest), which did not yield better results - probably due to the fact that these models are already ensembling methods.

**_Overfitting_**  

In the XGBoost case the model seems to overfit a little bit, but not significantly.

### B. Hyperparameters tuning

We performed Grid Search using the `GridSearchCV` command (the `RandomizedSearchCV`is too costly), which is still quite demanding computationaly speaking but allowed us to come up with an efficient set of parameters for our XGBoost, gaining **-0,17** in test score compared to default parameters. Below, our final model:

In [ ]:
XGB = xgb.XGBRegressor(colsample_bytree = 0.7, 
                learning_rate = 0.05, max_depth = 10, 
                min_child_weight = 4, n_estimators = 5000,
                nthread = 4, objective = 'reg:linear', 
                silent = 1, subsample = 0.7)

## 4. Conclusion

**_Answer to the problem_**

Using the model we built, an airline company would be able to predict passenger load on a given route and day. This could help them **allocate planes** (e.g. schedule more flights on highly-frequented routes), or tune their **pricing policy** (increase ticket prices on highly-frequented routes). Indirectly, it could also help them adjust their **resource management**, by allocating more resources (investments, additional staff, etc.) to the routes with the highest passenger load.

In addition, the predictions could be **further analysed** (for example grouped by week or month, comparde with other airline companies, etc.) in oder to produce additional insights about **what determines passenger traffic the most**, what are the **preferred dates**, the **preferred routes**... These additional analyses would very provide valuable information to the airline company in order to **help them improve business performance**.

**_What we learnt in this project_**

We confronted ourselves to the different steps of a data science project, from obtaining and pre-processing the data to choosing and tuning the model. We tried many different data manipulation techniques and explored several models along the way, with a constant back and forth between model tuning and features engineeering. We also learnt to use github in a team project.  

$\rightarrow$ This experience allowed us to apply what we learnt in class, and most of all it helped us learn a lot by exploring the internet (stack overflow, etc.) to find solutions to our problems, in a "learning by doing" approach!